In [6]:
import pandas as pd
import numpy as np

np.random.seed(42)

num_rows = 6000
num_columns = 100
pressure_columns = ['pressure' + str(i) for i in range(1, num_columns + 1)]

# Generate random pressure values between 0 and 4095 for each column
pressure_values = np.random.randint(0, 4096, size=(num_rows, num_columns))

# Count the number of pressure values less than 1000 in each row
pressure_below_threshold = np.sum(pressure_values < 1000, axis=1)

# Set sleep column based on the condition: if pressure_below_threshold < 40, sleep = 0, else sleep = 1
sleep_values = np.where(pressure_below_threshold < 40, 0, 1)

# Shuffle the sleep values to ensure half of them are 1 and half are 0
np.random.shuffle(sleep_values)

# Create DataFrame with pressure columns and sleep column
df = pd.DataFrame(pressure_values, columns=pressure_columns)
df['sleep'] = sleep_values

# Count the number of rows with sleep equal to 1
count_sleep_1 = np.sum(df['sleep'] == 1)

# Adjust sleep values if needed to ensure 3000 rows have sleep equal to 1
if count_sleep_1 < num_rows // 2:
    diff = (num_rows // 2) - count_sleep_1
    indices_sleep_0 = df[df['sleep'] == 0].index[:diff]
    df.loc[indices_sleep_0, 'sleep'] = 1

# Save DataFrame to CSV
df.to_csv('dummy_pressure_dataset.csv', index=False)

print(df.head())


   pressure1  pressure2  pressure3  pressure4  pressure5  pressure6  \
0       3174       3507        860       1294       1130       1095   
1        702       2449       2777       1579        161        201   
2       3646        863       2790       3312        563       1020   
3       3296       2432       3474       1661       1153       1076   
4       3991        627        586       1648       4039       1443   

   pressure7  pressure8  pressure9  pressure10  ...  pressure92  pressure93  \
0       3772       3092       1638        2169  ...        3459        3073   
1       1981        995       2317        3934  ...        2842         392   
2         95       1923       1757        2532  ...         698        2160   
3       1707       2777       1695        1733  ...        1682        2863   
4       1445        103       3923         253  ...        3420        2544   

   pressure94  pressure95  pressure96  pressure97  pressure98  pressure99  \
0        1021        

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import joblib

df = pd.read_csv('dummy_pressure_dataset.csv')
X = df.drop('sleep', axis=1)
y = df['sleep']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
clf = KNeighborsClassifier(n_neighbors=15)
clf.fit(X_train_scaled, y_train)
joblib.dump(clf, 'pressure_model2.pkl')


['pressure_model2.pkl']

In [11]:
import numpy as np
import joblib

# Load the trained KNeighborsClassifier model from the pickle file
clf = joblib.load('pressure_model2.pkl')

# Generate random test data for one row
num_columns = 100
test_data = np.random.randint(0, 4096, size=(1, num_columns))

# Make predictions on the test data
prediction = clf.predict(test_data)

# Print the predicted 'sleep' value for the test data
predicted_sleep = prediction[0]  # Assuming prediction is a NumPy array with one element
print("Predicted 'sleep' value for the test data:", predicted_sleep)


Predicted 'sleep' value for the test data: 0
